In [1]:
import warnings, sys, os, gc
from os.path import join
warnings.filterwarnings("ignore")
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# import torch; print(torch.cuda.is_available())

import libemg
from libemg.datasets import get_dataset_list
from libemg.feature_extractor import FeatureExtractor

import numpy as np, pandas as pd
import random, copy, time, csv, json
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

from utils import *
from Models.CNN import CNN

SEED = 13; random.seed(SEED); np.random.seed(SEED)
GENERATOR = torch.manual_seed(SEED)
MMAP_MODE = 'r'

Bluepy not installed...


In [2]:
from SPACESHIP import SPACESHIP
dataset = SPACESHIP(dataset_folder='spaceship')
data = dataset.prepare_data()

path = join(PATH, 'ssl')
os.makedirs(path, exist_ok=True)

np.save(join(path, 'ssl_data'), data)

ssl_windows, ssl_meta = data.parse_windows(SEQ, SSL_INC)

np.save(join(path, 'ssl_windows'), ssl_windows.astype(DTYPE))
np.save(join(path, 'ssl_meta'), ssl_meta)

ssl_windows.shape

Finished subject=0 | reps=6 | out=spaceship_PROCESSED/subject0.h5
Finished subject=1 | reps=12 | out=spaceship_PROCESSED/subject1.h5
Finished subject=2 | reps=6 | out=spaceship_PROCESSED/subject2.h5
Finished subject=3 | reps=5 | out=spaceship_PROCESSED/subject3.h5
Finished subject=4 | reps=6 | out=spaceship_PROCESSED/subject4.h5
Finished subject=5 | reps=4 | out=spaceship_PROCESSED/subject5.h5
Finished subject=6 | reps=6 | out=spaceship_PROCESSED/subject6.h5
Finished subject=7 | reps=6 | out=spaceship_PROCESSED/subject7.h5
Finished subject=8 | reps=6 | out=spaceship_PROCESSED/subject8.h5
Finished subject=9 | reps=6 | out=spaceship_PROCESSED/subject9.h5
Finished subject=10 | reps=6 | out=spaceship_PROCESSED/subject10.h5
Finished subject=11 | reps=6 | out=spaceship_PROCESSED/subject11.h5
Finished subject=12 | reps=5 | out=spaceship_PROCESSED/subject12.h5
Finished subject=13 | reps=6 | out=spaceship_PROCESSED/subject13.h5
Finished subject=14 | reps=6 | out=spaceship_PROCESSED/subject14.h5

(999836, 8, 200)

In [3]:
from gForceSGT import gForceSGT
dataset = gForceSGT(dataset_folder='data0_sgt')
data = dataset.prepare_data(trans_delay=200)

path = join(PATH, 'sgt')
os.makedirs(path, exist_ok=True)

np.save(join(path, 'sgt_data0'), data)

Processing dataset from data0_sgt to data0_sgt_PROCESSED...
Subject 0 processed. Output: data0_sgt_PROCESSED/S0.h5
Subject 1 processed. Output: data0_sgt_PROCESSED/S1.h5
Subject 2 processed. Output: data0_sgt_PROCESSED/S2.h5
Subject 3 processed. Output: data0_sgt_PROCESSED/S3.h5
Subject 4 processed. Output: data0_sgt_PROCESSED/S4.h5
Subject 5 processed. Output: data0_sgt_PROCESSED/S5.h5


In [4]:
from gForceSGT import gForceSGT
dataset = gForceSGT(dataset_folder='data1_sgt')
data = dataset.prepare_data(trans_delay=200)

path = join(PATH, 'sgt')
os.makedirs(path, exist_ok=True)

np.save(join(path, 'sgt_data1'), data)

Processing dataset from data1_sgt to data1_sgt_PROCESSED...
Subject 0 processed. Output: data1_sgt_PROCESSED/S0.h5
Subject 1 processed. Output: data1_sgt_PROCESSED/S1.h5
Subject 2 processed. Output: data1_sgt_PROCESSED/S2.h5
Subject 3 processed. Output: data1_sgt_PROCESSED/S3.h5
Subject 4 processed. Output: data1_sgt_PROCESSED/S4.h5


In [ ]:
from gForceSGT import gForceSGT
dataset = gForceSGT()
data = dataset.prepare_data()

train_data = data.isolate_data("subjects", list(range(4)), fast=True)
val_data = data.isolate_data("subjects", list(range(4, 5)), fast=True)
test_data = data.isolate_data("subjects", list(range(6)), fast=True)

path = join(PATH, 'sgt')
os.makedirs(path, exist_ok=True)

np.save(join(path, 'train_data'), train_data)
np.save(join(path, 'val_data'), val_data)
np.save(join(path, 'test_data'), test_data)

train_windows, train_meta = train_data.parse_windows(SEQ, INC)
val_windows, val_meta = val_data.parse_windows(SEQ, INC)
test_windows, test_meta = test_data.parse_windows(SEQ, INC)

np.save(join(path, 'train_windows'), train_windows.astype(DTYPE))
np.save(join(path, 'train_meta'), train_meta)

np.save(join(path, 'val_windows'), val_windows.astype(DTYPE))
np.save(join(path, 'val_meta'), val_meta)

np.save(join(path, 'test_windows'), test_windows.astype(DTYPE))
np.save(join(path, 'test_meta'), test_meta)

train_windows.shape, val_windows.shape, test_windows.shape

In [ ]:
path = join(PATH, 'ssl')
ssl_windows = np.load(join(path, 'ssl_windows.npy'), mmap_mode=MMAP_MODE)

path = join(PATH, 'sgt')
train_windows = np.load(join(path, 'train_windows.npy'), mmap_mode=MMAP_MODE)
train_meta = np.load(join(path, 'train_meta.npy'), allow_pickle=True).item()
val_windows = np.load(join(path, 'val_windows.npy'), mmap_mode=MMAP_MODE)
val_meta = np.load(join(path, 'val_meta.npy'), allow_pickle=True).item()
test_windows = np.load(join(path, 'test_windows.npy'), mmap_mode=MMAP_MODE)
test_meta = np.load(join(path, 'test_meta.npy'), allow_pickle=True).item()